# Load Qick

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from qick_workspace.tools import system_cfg
from qick_workspace.tools.system_tool import select_config_idx
from qick import *
from qick.pyro import make_proxy
from qick import QickConfig
from qick.asm_v2 import QickSpan, QickSweep1D
import Pyro4

Pyro4.config.SERIALIZER = "pickle"
Pyro4.config.PICKLE_PROTOCOL_VERSION = 4

ns_host = "192.168.10.63"
ns_port = 8888
proxy_name = "myqick"

soc, soccfg = make_proxy(ns_host=ns_host, ns_port=ns_port, proxy_name=proxy_name)
print(soccfg)

Pyro.NameServer PYRO:Pyro.NameServer@0.0.0.0:8888
myqick PYRO:obj_100127414b1446cf94bc54217e2af34c@192.168.10.63:43131
QICK running on ZCU216, software version 0.2.371

Firmware configuration (built Tue Sep 10 16:13:40 2024):

	Global clocks (MHz): tProc dispatcher timing 430.080, RF reference 245.760
	Groups of related clocks: [tProc timing clock, DAC tile 0, DAC tile 1, DAC tile 3], [DAC tile 2], [ADC tile 2]

	16 signal generator channels:
	0:	axis_signal_gen_v6 - fs=9584.640 Msps, fabric=599.040 MHz
		envelope memory: 16384 complex samples (1.709 us)
		32-bit DDS, range=9584.640 MHz
		DAC tile 2, blk 0 is 0_230 on JHC3, or QICK box DAC port 8
	1:	axis_signal_gen_v6 - fs=9584.640 Msps, fabric=599.040 MHz
		envelope memory: 4096 complex samples (0.427 us)
		32-bit DDS, range=9584.640 MHz
		DAC tile 2, blk 1 is 1_230 on JHC4, or QICK box DAC port 9
	2:	axis_signal_gen_v6 - fs=9584.640 Msps, fabric=599.040 MHz
		envelope memory: 8192 complex samples (0.855 us)
		32-bit DDS, range=9584.

# Load system configuration

In [2]:
qubit_idx = 0

config = {
    **system_cfg.hw_cfg,
    **system_cfg.readout_cfg,
    **system_cfg.qubit_cfg,
    **system_cfg.expt_cfg,
    **system_cfg.cool_cfg,
}

In [3]:
from qick_workspace.scrip.s002_res_spec_ge import Resonator_onetone
from qick_workspace.scrip.s003_qubit_spec_ge import Qubit_Twotone
from qick_workspace.scrip.s005_power_rabi_ge import Amp_Rabi
from qick_workspace.scrip.s006_Ramsey_ge import Ramsey
from qick_workspace.scrip.s007_SpinEcho_ge import SpinEcho
from qick_workspace.scrip.s008_T1_ge import T1

# Test experiment config setting

In [ ]:
config["nqz_qubit"] = 2
config["res_gain_ge"][qubit_idx] = 0.2
config["qubit_ch"] = 5
# Resonator One-tone Configuration
START_FREQ = config["res_freq_ge"][qubit_idx] - 10  # [MHz]
STOP_FREQ = config["res_freq_ge"][qubit_idx] + 10  # [MHz]
STEPS = 101

onetone_cfg = select_config_idx(config, qubit_idx)
onetone_cfg.update(
    [
        ("steps", STEPS),
        ("res_freq_ge", QickSweep1D("freqloop", START_FREQ, STOP_FREQ)),
        ("relax_delay", 1),
        ("cooling", False),
    ]
)

# Qubit Two-tone Configuration
center = 4600
SPAN = 100
START_FREQ = center - SPAN  # [MHz]
STOP_FREQ = center + SPAN  # [MHz]
STEPS = 201
run_cfg = select_config_idx(config, qubit_idx)
run_cfg.update(
    [
        ("steps", STEPS),
        ("qubit_freq_ge", QickSweep1D("freqloop", START_FREQ, STOP_FREQ)),
        ("qmixer_freq", center),
        ("qubit_gain_ge", 0.2),
        ("qubit_length_ge", 5),
        ("relax_delay", 1),
    ]
)

# Amp Rabi Configuration
START_GAIN = 0.0  # [DAC units]
STOP_GAIN = 1  # [DAC units]
STEPS = 100
config["sigma"][qubit_idx] = 0.05
config["relax_delay"] = 50  # [us]
run_cfg = select_config_idx(config, qubit_idx)
run_cfg.update(
    [
        ("steps", STEPS),
        ("qubit_gain_ge", QickSweep1D("gainloop", START_GAIN, STOP_GAIN)),
    ]
)

# Ramsey Configuration
START_TIME = 0.0  # [us]
STOP_TIME = 5  # [us]
STEPS = 100
run_cfg = select_config_idx(config, qubit_idx)
run_cfg.update(
    [
        ("steps", STEPS),
        ("wait_time", QickSweep1D("waitloop", START_TIME, STOP_TIME)),
        ("ramsey_freq", 2),
    ]
)
# Spin Echo Configuration
START_TIME = 0.0  # [us]
STOP_TIME = 10  # [us]
STEPS = 100
run_cfg.update(
    [
        ("steps", STEPS),
        ("wait_time", QickSweep1D("waitloop", START_TIME, STOP_TIME)),
        ("ramsey_freq", 1),
    ]
)

# T1 Configuration
run_cfg = select_config_idx(config, qubit_idx)

START_TIME = 0.0  # [us]
STOP_TIME = 100  # [us]
STEPS = 100
run_cfg.update(
    [
        ("steps", STEPS),
        ("wait_time", QickSweep1D("waitloop", START_TIME, STOP_TIME)),
        ("relax_delay", 50),
        ("cooling", False),
        ("qubit_ch", 7),
    ]
)

In [ ]:
qubit_idx = 0
pyavg = 20

from qick_workspace.tools import fitting as fitter


def process_fitting(xpts, data, fitfunc):
    data = {
        "xpts": xpts,
        "amps": np.abs(data),
        "phase": np.unwrap(np.angle(data)),
        "avgi": data.real,
        "avgq": data.imag,
    }

    for measure in ("amps", "phase", "avgi", "avgq"):
        popt, pcov, f = fitfunc(data["xpts"], data[measure])
        data[f"fit_{measure}"] = popt
        data[f"fit_err_{measure}"] = pcov

    fit_params, fit_err, best_measure = fitter.get_best_fit(data, fitfunc=None)

    return fit_params, fit_err, best_measure


def fix_phase(p) -> float:
    """
    Normalize phase and calculate pi gain.

    Args:
        p: Parameters list containing phase information

    Returns:
        Pi gain value
    """
    if p[2] > 180:
        p[2] = p[2] - 360
    elif p[2] < -180:
        p[2] = p[2] + 360

    if p[2] < 0:
        pi_gain = (1 / 2 - p[2] / 180) / 2 / p[1]
        pi2_gain = (0 - p[2] / 180) / 2 / p[1]
    else:
        pi_gain = (3 / 2 - p[2] / 180) / 2 / p[1]
        pi2_gain = (1 - p[2] / 180) / 2 / p[1]
    return pi_gain, pi2_gain


class auto:
    def __init__(self, cfg) -> None:
        self.cfg = cfg
        self.cfg["nqz_qubit"] = 2
        self.cfg["res_gain_ge"][qubit_idx] = 0.2
        self.cfg["qubit_ch"] = 5
        self.cfg["sigma"][qubit_idx] = 0.005
        self.report = {}

    def auto_onetone(self):
        # Resonator One-tone Configuration
        START_FREQ = self.cfg["res_freq_ge"][qubit_idx] - 10  # [MHz]
        STOP_FREQ = self.cfg["res_freq_ge"][qubit_idx] + 10  # [MHz]
        STEPS = 101

        onetone_cfg = select_config_idx(self.cfg, qubit_idx)
        onetone_cfg.update(
            [
                ("steps", STEPS),
                ("res_freq_ge", QickSweep1D("freqloop", START_FREQ, STOP_FREQ)),
                ("relax_delay", 1),
            ]
        )
        res_onetone = Resonator_onetone(soc, soccfg, onetone_cfg)
        res_onetone.auto(py_avg=pyavg)
        ## update value ##
        fres = res_onetone.plot_circle()
        self.cfg["res_freq_ge"][qubit_idx] = round(fres[0] / 1e6, 4)
        print(
            "Updated Resonator Frequency: ", self.cfg["res_freq_ge"][qubit_idx], " MHz"
        )
        self.report["resonator_frequency_MHz"] = self.cfg["res_freq_ge"][qubit_idx]

    def auto_twotone(self):
        # Qubit Two-tone Configuration
        center = 4600
        SPAN = 100
        START_FREQ = center - SPAN  # [MHz]
        STOP_FREQ = center + SPAN  # [MHz]
        STEPS = 201
        run_cfg = select_config_idx(self.cfg, qubit_idx)
        run_cfg.update(
            [
                ("steps", STEPS),
                ("qubit_freq_ge", QickSweep1D("freqloop", START_FREQ, STOP_FREQ)),
                ("qmixer_freq", center),
                ("qubit_gain_ge", 0.2),
                ("qubit_length_ge", 5),
                ("relax_delay", 1),
            ]
        )
        spectrum_ge = Qubit_Twotone(soc, soccfg, run_cfg)
        spectrum_ge.auto(py_avg=pyavg)
        freq, iq = spectrum_ge.freqs, spectrum_ge.iqdata
        pOpt, pCov, fitparams = process_fitting(freq, iq, fitter.fitlor)
        self.cfg["qubit_freq_ge"][qubit_idx] = round(pOpt[2], 6)
        self.cfg["qmixer_freq"][qubit_idx] = round(pOpt[2], 6)
        print("Updated Qubit Frequency: ", self.cfg["qubit_freq_ge"][qubit_idx], " MHz")
        self.report["qubit_frequency_MHz"] = self.cfg["qubit_freq_ge"][qubit_idx]

    def auto_rabi(self):
        # Amp Rabi Configuration
        START_GAIN = 0.0  # [DAC units]
        STOP_GAIN = 1  # [DAC units]
        STEPS = 100
        self.cfg["relax_delay"] = 50  # [us]

        run_cfg = select_config_idx(self.cfg, qubit_idx)
        current_sigma = self.cfg["sigma"][qubit_idx]
        run_cfg.update(
            [
                ("steps", STEPS),
                ("sigma", current_sigma),
                ("qubit_gain_ge", QickSweep1D("gainloop", START_GAIN, STOP_GAIN)),
            ]
        )

        prabi = Amp_Rabi(soc, soccfg, run_cfg)
        prabi.auto(py_avg=pyavg)
        pOpt, pCov, fitparams = process_fitting(
            prabi.gains, prabi.iqdata, fitter.fitdecaysin
        )

        pi_gain, pi2_gain = fix_phase(pOpt)
        self.report["rabi_pi_gain"] = pi_gain
        self.report["rabi_pi2_gain"] = pi2_gain
        self.report["sigma"] = current_sigma
        return pi_gain, pi2_gain

    def auto_rabi_loop(self, max_iterations=5):
        pi_gain = 2.0
        iteration = 0

        while pi_gain > 0.9 and iteration < max_iterations:
            iteration += 1
            current_sigma = self.cfg["sigma"][qubit_idx]
            print(
                f"\n[{iteration}/{max_iterations}] Running Rabi. Current sigma: {current_sigma:.4f}"
            )

            pi_gain, pi2_gain = self.auto_rabi()

            if pi_gain > 0.9:
                old_sigma = self.cfg["sigma"][qubit_idx]
                new_sigma = old_sigma + 0.01

                self.cfg["sigma"][qubit_idx] = round(new_sigma, 4)

                print(
                    f"  ❌ Pi Gain ({pi_gain:.4f}) too high. Sigma -> {new_sigma:.4f}"
                )
            else:
                self.cfg["qubit_pi_gain_ge"][qubit_idx] = pi_gain
                self.cfg["qubit_pi2_gain_ge"][qubit_idx] = pi2_gain

                print(f"  ✅ Pi Gain ({pi_gain:.4f}) OK. Loop finished.")

        if pi_gain > 0.9 and iteration == max_iterations:
            print("\n⚠️ WARNING: Max iterations reached. Pi gain still > 0.9.")

    def auto_ramsey(self):
        START_TIME = 0.0  # [us]
        STOP_TIME = 5  # [us]
        STEPS = 100
        run_cfg = select_config_idx(self.cfg, qubit_idx)
        run_cfg.update(
            [
                ("steps", STEPS),
                ("wait_time", QickSweep1D("waitloop", START_TIME, STOP_TIME)),
                ("ramsey_freq", 2),
            ]
        )
        ramsey = Ramsey(soc, soccfg, run_cfg)
        ramsey.auto(py_avg=pyavg)
        pOpt, pCov, fitparams = process_fitting(
            ramsey.delay_times, ramsey.iqdata, fitter.fitdecaysin
        )
        self.report["T2_ramsey_us"] = pOpt[3]
        return pOpt

    def auto_finetune_loop(self, max_iterations=3):
        DETUNE_THRESHOLD = 0.005
        iteration = 0
        current_detune = DETUNE_THRESHOLD + 1.0

        print("\n--- Starting Qubit Frequency Fine-Tuning Loop ---")

        while abs(current_detune) > DETUNE_THRESHOLD and iteration < max_iterations:
            iteration += 1
            print(f"[{iteration}/{max_iterations}] Running Ramsey to measure detune...")

            pOpt = self.auto_ramsey()
            current_detune = pOpt[1] - self.cfg["ramsey_freq"]

            if abs(current_detune) > DETUNE_THRESHOLD:
                adjustment = round(current_detune, 2)

                old_freq = self.cfg["qubit_freq_ge"][qubit_idx]
                new_freq = old_freq - adjustment

                self.cfg["qubit_freq_ge"][qubit_idx] = new_freq

                print(
                    f"Detune: {current_detune:.5f} MHz. Adjusting freq: {old_freq:.5f} MHz -> {new_freq:.5f} MHz"
                )
            else:
                print(
                    f"Detune: {current_detune:.5f} MHz (< {DETUNE_THRESHOLD} MHz). Loop finished."
                )
                break

        if abs(current_detune) > DETUNE_THRESHOLD and iteration == max_iterations:
            print(
                f"\nWARNING: Max iterations reached. Final detune is {current_detune:.5f} MHz."
            )

        return self.cfg["qubit_freq_ge"][qubit_idx]

    def auto_echo(self):
        START_TIME = 0.0  # [us]
        STOP_TIME = 5  # [us]
        STEPS = 100
        run_cfg = select_config_idx(self.cfg, qubit_idx)
        run_cfg.update(
            [
                ("steps", STEPS),
                ("wait_time", QickSweep1D("waitloop", START_TIME, STOP_TIME)),
            ]
        )
        echo = SpinEcho(soc, soccfg, run_cfg)
        echo.auto(py_avg=pyavg)
        pOpt, pCov, fitparams = process_fitting(
            echo.delay_times, echo.iqdata, fitter.fitdecaysin
        )

        print("Updated Qubit T2 Echo: ", round(pOpt[3], 3), " us")
        self.report["T2_echo_us"] = pOpt[3]
        return pOpt

    def auto_t1(self):
        START_TIME = 0.0  # [us]
        STOP_TIME = 100  # [us]
        STEPS = 100
        run_cfg = select_config_idx(self.cfg, qubit_idx)
        run_cfg.update(
            [
                ("steps", STEPS),
                ("wait_time", QickSweep1D("waitloop", START_TIME, STOP_TIME)),
            ]
        )

        t1_exp = T1(soc, soccfg, run_cfg)
        t1_exp.auto(py_avg=pyavg)
        pOpt, pCov, fitparams = process_fitting(
            t1_exp.delay_times, t1_exp.iqdata, fitter.fitexp
        )
        self.report["T1_us"] = pOpt[2]
        print("Updated Qubit T1: ", round(pOpt[2], 3), " us")

    def auto_report(self):
        print("Auto calibration report:")
        print(f"Resonator Frequency {self.report['resonator_frequency_MHz']:.3f} MHz")
        print(f"Qubit Frequency {self.report['qubit_frequency_MHz']:.3f} MHz")
        print(f"Rabi Pi Gain {self.report['rabi_pi_gain']:.3f}")
        print(f"Rabi Pi/2 Gain {self.report['rabi_pi2_gain']:.3f}")
        print(f"Sigma {self.report['sigma']:.3f} us")
        print(f"T2 Ramsey {self.report['T2_ramsey_us']:.3f} us")
        print(f"T2 Echo {self.report['T2_echo_us']:.3f} us")
        print(f"T1 {self.report['T1_us']:.3f} us")

In [28]:
from time import time

start = time()
cal = auto(config)
cal.auto_onetone()
cal.auto_twotone()
cal.auto_rabi_loop(max_iterations=10)
cal.auto_finetune_loop(max_iterations=10)
cal.auto_rabi()
cal.auto_echo()
cal.auto_t1()
cal.auto_report()
end = time()
print(f"\nTotal auto calibration time: {end - start:.2f} seconds")

Updated Qubit T1:  7.092  us
Auto calibration report:
Resonator Frequency 5351.583 MHz
Qubit Frequency 4635.173 MHz
Rabi Pi Gain 0.754
Rabi Pi/2 Gain 0.377
Sigma 0.025
T2 Ramsey (us) 3.787
T2 Echo (us) 5.098
T1 (us) 7.092

Total auto calibration time: 146.61 seconds
